# 04 - So sánh với Scikit-learn RandomForest và SMOTE

## 1. Cài đặt Thư viện

Bước đầu tiên là cài đặt các thư viện cần thiết, bao gồm `imbalanced-learn` để sử dụng SMOTE.

## 2. Tải Dữ liệu và Chuẩn bị

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt

# Tải dữ liệu đã xử lý
train_df = pd.read_csv('../data/processed/aug_train_processed.csv')
test_df = pd.read_csv('../data/processed/aug_test_processed.csv')

print("Tải dữ liệu thành công.")
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

## 3. Xử lý Mất cân bằng với SMOTE

Tách dữ liệu và áp dụng SMOTE **chỉ trên tập huấn luyện** để tạo ra các mẫu dữ liệu tổng hợp cho lớp thiểu số, giúp cân bằng bộ dữ liệu.

In [ ]:
# Tách features (X) và target (y)
X = train_df.drop('target', axis=1)
y = train_df['target']

# Chia dữ liệu: 80% train, 20% validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Kích thước trước khi SMOTE: X_train shape={X_train.shape}")
print(f"Tỷ lệ lớp 1 ban đầu trong y_train: {y_train.mean():.2f}")

# Áp dụng SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(f"\nKích thước sau khi SMOTE: X_train shape={X_train_smote.shape}")
print(f"Tỷ lệ lớp 1 sau đó trong y_train_smote: {y_train_smote.mean():.2f}")

## 4. Huấn luyện Model `RandomForestClassifier`

Chúng ta sẽ huấn luyện một mô hình `RandomForestClassifier` từ thư viện `scikit-learn` trên dữ liệu đã được xử lý bằng SMOTE.

In [ ]:
# Khởi tạo và huấn luyện model
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)

print("Bắt đầu huấn luyện scikit-learn RandomForest...")
rf_clf.fit(X_train_smote, y_train_smote)
print("Huấn luyện hoàn tất.")

## 5. Đánh giá Model

Chúng ta sẽ đánh giá hiệu suất của mô hình trên cả tập train (đã SMOTE) và validation (dữ liệu gốc, không cân bằng) để có cái nhìn toàn diện.

In [ ]:
def evaluate_and_print(model, X_train, y_train, X_val, y_val):
    # Dự đoán trên tập train
    y_pred_train = model.predict(X_train)
    print("--- Đánh giá trên tập HUẤN LUYỆN (đã SMOTE) ---")
    print(classification_report(y_train, y_pred_train))

    # Dự đoán trên tập validation
    y_pred_val = model.predict(X_val)
    print("\n--- Đánh giá trên tập XÁC THỰC ---")
    print(classification_report(y_val, y_pred_val))

    # --- ROC AUC ---
    # Lấy xác suất dự đoán cho lớp dương (lớp 1)
    y_proba_val = model.predict_proba(X_val)[:, 1]
    
    # Tính FPR, TPR và ngưỡng
    fpr, tpr, thresholds = roc_curve(y_val, y_proba_val)
    
    # Tính diện tích dưới đường cong (AUC)
    roc_auc = auc(fpr, tpr)
    print(f"\nROC AUC Score (Validation): {roc_auc:.4f}")

    # --- Visualizations ---
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Ma trận nhầm lẫn
    cm = confusion_matrix(y_val, y_pred_val)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Looking', 'Looking'],
                yticklabels=['Not Looking', 'Looking'], ax=axes[0])
    axes[0].set_title('Confusion Matrix')
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')

    # Đường cong ROC
    axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')
    axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
    axes[1].set_xlim([0.0, 1.0])
    axes[1].set_ylim([0.0, 1.05])
    axes[1].set_xlabel('False Positive Rate')
    axes[1].set_ylabel('True Positive Rate')
    axes[1].set_title('ROC Curve & AUC Score')
    axes[1].legend(loc="lower right")
    
    plt.tight_layout()
    plt.show()

evaluate_and_print(rf_clf, X_train_smote, y_train_smote, X_val, y_val)

## 6. Kết luận Baseline

Kết quả này từ một thư viện tiêu chuẩn như `scikit-learn` cùng với SMOTE sẽ là thước đo (baseline) hiệu suất của chúng ta. Nếu mô hình tự triển khai trong notebook `03` có kết quả gần với baseline này, điều đó cho thấy việc triển khai của chúng ta là chính xác và hiệu quả.